# Prelude

In [ ]:
# Enable interactive plots (%matplotlib -l to list backends)
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np


from bgcellmodels.common import analysis
from bgcellmodels.mechanisms import noise, synapses

import neuron; h = neuron.h
import bluepyopt.ephys as ephys

# print code version (hash of checked out version)
print("\nCurrent commit:")
!git log -1
print("\nChanges since last commit:")
!git status --short

In [ ]:
# Load cell model
from bgcellmodels.models.striatum.Mahon2000_MSN import mahon_pynn_model
icell = h.MahonMSN()
soma = icell.soma[0]

In [ ]:
# Deactivate tables for testing
use_tables = 0
with_tables = ['Nam', 'Km', 'Kirm', 'KAfm', 'KAsm', 'Krpm', 'NaPm', 'NaSm']
without_tables = ['Nam', 'Km', 'Kirm', 'KAfm', 'KAsm', 'Krpm', 'NaPm', 'NaSm']

for mechname in without_tables:
    setattr(h, 'usetable_' + mechname, 0)

for mechname in with_tables:
    setattr(h, 'usetable_' + mechname, 1)

# Plot Rates

In [ ]:
# Rates and steady state values for Nam channel

phi = 5.
Vam = -28.
Kam = 1.
Vbm = -53. #NOT the original value from wang and Buzsaki
Kbm = 18.
Vah = -51. #NOT the original value from wang and Buzsaki
Kah = 20.
Vbh = -21. #NOT the original value from wang and Buzsaki
Kbh = 1.

v = np.linspace(-100, 100, 400)
exp = np.exp

am = (-0.1*(v-Vam)/Kam / (exp(-0.1 * (v-Vam)/Kam) - 1))
Bm = 4 * exp(-(v-Vbm)/Kbm)
ah = 0.07 * exp(-(v-Vah)/Kah)
Bh =  1 / (1 + exp(-0.1*(v-Vbh)/Kbh))

fig, ax = plt.subplots()
ax.plot(v, am, label='am')
ax.plot(v, Bm, label='Bm')
ax.plot(v, ah, label='ah')
ax.plot(v, Bh, label='Bh')
ax.set_xlim((-100,100))
ax.legend()

fig, ax = plt.subplots()
ax.plot(v, am/(am+Bm), label='minf')
ax.plot(v, ah/(ah+Bh), label='hinf')
ax.set_xlim((-100,100))
ax.legend()

# hinf/tauh = 
hinf = ah/(ah+Bh)
tauh = 1.0 / (phi * (ah+Bh))

fig, ax = plt.subplots()
ax.plot(v, ah+Bh, label='ah+Bh')
ax.plot(v, tauh, label='tauh')
ax.plot(v, hinf/tauh, label='hinf/tauh')
ax.set_xlim((-100,100))
ax.legend()

# Recording

In [ ]:
rec_secs = {
    'soma': soma,
}

trace_specs = {
    # Membrane voltages
    'V_soma': {'var':'v', 'sec':'soma', 'loc':0.5},
}

rec_dt = 0.05
vec_dict, markers = analysis.recordTraces(rec_secs, trace_specs, rec_dt)

# Custom recordings
vrec = h.Vector()
vrec.record(soma(0.5)._ref_v)
trec = h.Vector()
trec.record(h._ref_t)

# Protocol: Spontaneous Activity

In [ ]:
# Init and run simulation
h.dt = 0.025
h.celsius = 37.0
h.v_init = -77.4 # from Mahon article code
h.tstop = 1000.0
h.init()
h.run()

In [ ]:
v_soma = vec_dict['V_soma'].as_numpy()
t_rec = np.arange(len(v_soma)) * rec_dt

plt.figure()
plt.plot(t_rec, v_soma)
plt.xlim(0, h.tstop)
plt.ylim(-80, 60)
plt.xlabel('time (ms)')
plt.ylabel('$V_m$ (mV)')
plt.title('Spontaneous activity')

# Protocol: Fig 2A

In [ ]:
# Fig 2A: create an electrode in the soma that creates two current pulses (200 ms in duration) 200ms apart
stim1 = h.IClamp(soma(0.5))
stim1.amp = 0.00167 * 1.7 # 0.00167
stim1.delay = 100
stim1.dur = 200

stim2 = h.IClamp(soma(0.5))
stim2.amp = 0.00167 * 1.7
stim2.delay = 500
stim2.dur = 200

In [ ]:
# Init and run simulation
h.dt = 0.025
h.celsius = 37.0
h.v_init = -77.4 # from Mahon article code
h.tstop = 1000.0
h.init()
h.run()

In [ ]:
v_soma = vec_dict['V_soma'].as_numpy()
t_rec = np.arange(len(v_soma)) * rec_dt

plt.figure()
plt.plot(t_rec, v_soma)
plt.xlim(0, h.tstop)
plt.ylim(-100, 60)
plt.xlabel('time (ms)')
plt.ylabel('$V_m$ (mV)')
plt.title('Delayed tonic firing (Fig. 2A)')

# Protocol: Fig 2B

In [ ]:
soma.gkd_KdFSI = 0.0018
stim.amp = 0.0042

In [ ]:
# Init and run simulation
h.dt = 0.025
h.celsius = 37.0
h.v_init = -77.4 # from Mahon article code
h.tstop = 1000.0
h.init()
h.run()

In [ ]:
v_soma = vec_dict['V_soma'].as_numpy()
t_rec = np.arange(len(v_soma)) * rec_dt

plt.figure()
plt.plot(t_rec, v_soma)
plt.xlim(0, h.tstop)
plt.ylim(-100, 60)
plt.xlabel('time (ms)')
plt.ylabel('$V_m$ (mV)')
plt.title('Tonic stuttering (Fig. 2B)')